In [1]:
#필요 모듈 임포트
from google.cloud import texttospeech
from google.cloud import speech
from pydub import AudioSegment
from konlpy.tag import Okt
from konlpy.tag import Kkma
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
import io

# 1. Google Cloud Text-To-Speech API 사용하기

In [299]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './tts_key.json' #경로 자기에 맞게 설정

def synthesize_text(text):
    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    voice = texttospeech.VoiceSelectionParams(
        language_code='ko-KR',
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16)

    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config)

    with open('output.wav', 'wb') as out:
        out.write(response.audio_content)

In [300]:
#text to speech 실행하기
synthesize_text("삼성전자에 대해 알려줘.")

# 2. Google Cloud Speech-To-Text API 사용하기

## 2-1. 음성 파일 wav파일로 변환하기

In [301]:
m4a_file = './019.m4a'
wav_filename = './019.wav'
track = AudioSegment.from_file(m4a_file,  format= 'm4a')
file_handle = track.export(wav_filename, format='wav')

## 2-2. 음성파일 텍스트로 변환하기

In [302]:
client = speech.SpeechClient.from_service_account_json('./stt_key.json')

def transcribe_file(speech_file):
    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)

# 파일 인코딩 형식에 맞게 그때 그때 변경해주어야 함
    config = speech.RecognitionConfig(
#         encoding='LINEAR16',
#         sample_rate_hertz=44100, 
        language_code='ko-KR',
        model='default',
        enable_word_time_offsets=True,
        enable_automatic_punctuation=True
    )

    response = client.recognize(config=config, audio=audio)

    for result in response.results:
        print('{}'.format(result.alternatives[0].transcript))
        return result.alternatives[0].transcript

In [303]:
text = transcribe_file('./019.wav')

삼성전자에 대해서 알려 줘.


## 2-3. 인풋으로 받은 텍스트에서 주식 종목 추출하기

In [304]:
#kospi 상장 기업 목록 가져오기
krx = pd.read_csv('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/02. 대외활동/03. 신한AI 공모전/01. 소스/01. 데이터/KRX_data.csv')
companies = krx['Name'].tolist()

In [305]:
kkma = Kkma()

In [306]:
nouns = kkma.nouns(text)
print('Kkma 명사 추출 :', nouns)

Kkma 명사 추출 : ['삼성', '삼성전자', '전자']


In [307]:
company = None
found = "no"

for n in nouns:
    if n in companies:
        company = n
        found = "yes"
        break
        
if found == "no" :
    print("Company not here.")

In [308]:
company

'삼성전자'